### 0-导语
汇总：<br>
每一个算法——简单的概括介绍（功能运用/原理概述）+代码示例

### 8-降维
-1-主成分分析（PCA）：通过奇异值分解（SVD）得到主成分矩阵，提取前d个主成分得到降维矩阵

In [ ]:
# 以下代码应用 PCA 将数据集的维度降至两维（请注意，它会自动处理数据的中心化）
from sklearn.decomposition import PCA

# 维度/方差比率
pca=PCA(n_components=2/0.95)
X_reduced=pca.fit_transform(X)

-1--PCA变种：针对大数据集、多特征

In [ ]:
# 增量 PCA（Incremental PCA）：将训练集分批，并一次只对一个批量使用 IPCA 算法。这对大型训练集非常有用。
from sklearn.decomposition import IncrementalPCA
# 下面的代码将 MNIST 数据集分成 100 个小批量（使用 NumPy 的array_split()函数）
n_batches=100
# 并将它们提供给 Scikit-Learn 的IncrementalPCA类，以将 MNIST 数据集的维度降低到 154 维
inc_pca=IncrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist,n_batches):
    # 请注意，您必须对每个最小批次调用partial_fit()方法，而不是对整个训练集使用fit()方法
    inc_pca.partial_fit(X_batch)
X_mnist_reduced=inc_pca.transform(X_mnist)


# 随机 PCA（Randomized PCA）
# 计算复杂度是O(m × d^2) + O(d^3)，而不是O(m × n^2) + O(n^3)，所以当d远小于n时，它比之前的算法快得多。
rnd_pca=PCA(n_components=154,svd_solver='randomized')
X_reduced_rnd=rnd_pca.fit_transform(X_mnist)
X_reduced_rnd[0,:]

-2-核 PCA（Kernel PCA）:核技巧

In [ ]:
# 下面的代码使用 Scikit-Learn 的KernelPCA类来执行带有 RBF 核的 kPCA
from sklearn.decomposition import KernelPCA

rbf_pca=KernelPCA(n_components=2,kernel='rbf',gamma=0.04)
X_reduced=rbf_pca.fit_transform(X)
# 使用交叉验证的方格搜索来寻找可以最小化重建前图像误差的核方法和超参数

-3-LLE 局部线性嵌入（Locally Linear Embedding）<br>
首先测量每个训练实例与其最近邻（c.n.）之间的线性关系，然后寻找能最好地保留这些局部关系的训练集的低维表示

In [ ]:
# 以下代码使用 Scikit-Learn 的LocallyLinearEmbedding类来展开瑞士卷。得到的二维数据集如图 8-12 所示。
from sklearn.datasets import make_swiss_roll
# 三维坐标，标签值
X_sr,t = make_swiss_roll(n_samples=500,  noise=0.0, random_state=None) 

from sklearn.manifold import LocallyLinearEmbedding
lle=LocallyLinearEmbedding(n_components=2,n_neighbors=10)
X_reduced=lle.fit_transform(X_sr)

-4-基于Scikit-Learn的其他降维方法：<br>
聚类型：多维缩放（MDS）、Isomap、t-分布随机邻域嵌入（t-Distributed Stochastic Neighbor Embedding，t-SNE）<br>
线性投影型：线性判别分析（Linear Discriminant Analysis，LDA）<br>
t-SNE介绍：https://blog.csdn.net/haoji007/article/details/94962952#%E4%BB%A3%E7%A0%81

奇异值分解（SVD）：一种标准矩阵分解技术<br>
可以将训练集矩阵X（m×n，m：样本量，n：特征量）分解为三个矩阵U·Σ·V^T的点积，其中V^T（n×n）包含我们想要的所有主成分
https://blog.csdn.net/qq_42902997/article/details/109532508

t-SNE原理<br>
https://bindog.github.io/blog/2016/06/04/from-sne-to-tsne-to-largevis/

#### 思考
基于信息量的降维方法
方差最大原则/投影距离最小化原则：最小化信息损失，最大程度保留原始信息——信息损失最小化原则<br>
根据这一原则构造正交的信息量梯度<br>
<br>
核PCA最优参数搜索方法1：<br>
灵活性/具体化：针对不同的后续任务，采用最合适的降维方式；但有可能会过拟合<br>
核PCA最优参数搜索方法2：<br>
遵循最小化信息损失原则；不容易过拟合<br>